In [1]:
import numpy as np
from termcolor import colored

# Find all legal moves on a chessboard

I was asked this at a preliminary test for a hedge fund.

You have a chessboard where you can move from up left to down right boxes, only going right or down. Some boxes cannot be occupied, so some paths are illegal. Find a pseudocode that formulates all legal paths.

## Preliminaries

I give some parameters and a nice chessboard drawing function. The size of the chessboard dictates the structure of a path. Say we have a $4\times4$ chessboard, then the paths are composed by 3 moves to right, and 3 downwards. Clearly, paths consist in sequences like $\{rrdrdd\}$, that means a _right right down right down down_ path. One may see it just as the sequence $\{1,2,4\}$, that is the moves when one moves right. All the others will necessarily be downwards.

Here I just set the size, get the number of moves to the right, and build a nice chessboard drawing function for a given path.

In [2]:
size = 4

r_move = size - 1
moves = 2*r_move

def disegna_scacchiera(vietati,mosse,size,board_only = False):
    posizione=[1,1]
    passate = [ posizione.copy() ]
    for i in range(1,moves+1):
        if i in mosse:
            posizione[1] += 1
        else:
            posizione[0] += 1
        passate.append(posizione.copy())

    posizione=[1,1]
    for riga in range(size):
        for colonna in range(size):
            print('+---',end='')
        print('+')
        for colonna in range(size):
            if (posizione in passate) and (posizione in vietati):
                char = colored('X', 'red')
                if board_only:
                    char = 'X'
            elif posizione in passate:
                char = colored('o', 'red')
                if board_only:
                    char = ' '
            elif posizione in vietati:
                char = 'X'
            else:
                char = ' '
            print('| {} '.format(char),end='')
            posizione[1] += 1
        print('|')
        posizione[1] = 1
        posizione[0] += 1
    for colonna in range(size):
        print('+---',end='')
    print('+')

I need to compose the full set of moves, regardless than illegal tiles. I'll exclude them later. I can think it this way:

0. start from the elementary path made by moving right till you hit the right boundary, so basically $\{1,..,s-1\}$, for a $s\times s$ chessboard.
1. Take the last element and add 1. When it hits $2s-1$ increase by one the before-the-last element and set the last one to be the same value plus one. Example:
$$\{1,2,3\},\{1,2,4\},\{1,2,5\},\{1,2,6\},\{1,3,4\},\{1,3,5\},\{1,3,6\},\{1,4,5\},\{1,4,6\},\{1,5,6\},\{2,3,4\},..$$

The method is a bit annoying under a coding point of view. One can reformulate it to generate:
$$\{1,1,1\},\{1,1,2\},\{1,1,3\},\{1,1,4\},\{1,2,2\},\{1,2,3\},\{1,2,4\},\{1,3,3\},\{1,3,4\},\{1,4,4\},\{2,2,2\},..$$
and then add $\{0,1,2\}$ to each vector. The method must be suitably reformulated for each possible size indeed, but this is to show.

This is what ``compose_moves`` does. ``list_moves`` is the list carrying the moves of a single path, and indeed starts from the elementary one $\{1,..,1\}$.

The core loop does just this. Takes the last path, increases by one the last element, and then sweep backwards. If the $i$-th (pythonwise backwards) element is equal to $s+1$ (turn of dozen-like idea), then increases by one the preceeding one (pythonwise the $-(i+1)$-th element). Then, sweeps the list forwards. If an element is equal to $s+1$, must be set equal to the preceeding one. This is equivalent to manipulating the set of moves equivalent to $\{1,4,4\}$ as:
$$\{1,4,4\}\Rightarrow\{1,4,5\}\rightarrow\{1,5,5\}\rightarrow\{2,5,5\}\rightarrow\{2,2,5\}\rightarrow\{2,2,2\},$$
that is the last passage of the sequence above.

Then, ``compose_moves`` adds the mentioned $\{0,..,s-2\}$ to each path and returns the list of all paths, conveniently as a numpy array.

In [3]:
def compose_moves(size):
    list_moves = [1 for i in range(1,size)]
    all_paths = []

    while True:    
        all_paths.append(list_moves.copy())

        list_moves[-1] += 1
        for i in range(1,size-1):
            if list_moves[-i] == size+1:
                list_moves[-(i+1)] += 1
        for i in range(1,size-1):
            if list_moves[i] == size+1:
                list_moves[i] = list_moves[i-1]
        if list_moves[0] == size+1:
            break

    all_paths = np.array(all_paths+np.array([i for i in range(0,size-1)]))
    
    return all_paths

the ``find_forbidden_paths`` does what its name says. First of all formulates all possible paths. To find if a path is doable, thinks this way: take a forbidden tile in $(a,b)$. It takes $b-1$ moves to the right before the first $a+b-2$ in order to be hit. Imagine if $(3,2)$ is forbidden. One has to go twice down and once to the right to reach it. So $3+2-2=3$ moves, of which $2-1=1$ to the right. 

Take all paths. If a path has $b-1$ moves to the right before the move number $a+b-2$, it has to be excluded.

In [4]:
def find_forbidden_paths(forbidden_tiles,size):
    all_paths = compose_moves(size)
    forbidden_tiles = [ np.array(i) for i in forbidden_tiles ]

    start = np.array([1,1])
    forbidden_paths = []
    legal_paths = []

    for j in all_paths:
        forbidden = False
        for forbidden_tile in forbidden_tiles:
            r_moves = (forbidden_tile - start)[1]
            t_moves = (forbidden_tile - start).sum()
            if (j <= t_moves).sum() == r_moves:
                forbidden_paths.append(tuple(j))
                forbidden = True
        if not forbidden:
            legal_paths.append(tuple(j))

    return np.unique(legal_paths,axis = 0), np.unique(forbidden_paths,axis = 0)

In [5]:
all_paths = compose_moves(size)

forbidden_tiles = [ [2,1], [2,2], [2,3] ]

legal_paths, forbidden_paths = find_forbidden_paths(forbidden_tiles,size)

print(len(all_paths),len(forbidden_paths),len(legal_paths))

print('forbidden paths')
for i in forbidden_paths:
    disegna_scacchiera(forbidden_tiles,i,size,False)
    
print('legal paths')
for i in legal_paths:
    disegna_scacchiera(forbidden_tiles,i,size,False)

20 19 1
forbidden paths
+---+---+---+---+
| o | o | o |   |
+---+---+---+---+
| X | X | X | o |
+---+---+---+---+
|   |   |   | o |
+---+---+---+---+
|   |   |   | o |
+---+---+---+---+
+---+---+---+---+
| o | o | o |   |
+---+---+---+---+
| X | X | X |   |
+---+---+---+---+
|   |   | o | o |
+---+---+---+---+
|   |   |   | o |
+---+---+---+---+
+---+---+---+---+
| o | o | o |   |
+---+---+---+---+
| X | X | X |   |
+---+---+---+---+
|   |   | o |   |
+---+---+---+---+
|   |   | o | o |
+---+---+---+---+
+---+---+---+---+
| o | o |   |   |
+---+---+---+---+
| X | X | X | o |
+---+---+---+---+
|   |   |   | o |
+---+---+---+---+
|   |   |   | o |
+---+---+---+---+
+---+---+---+---+
| o | o |   |   |
+---+---+---+---+
| X | X | X |   |
+---+---+---+---+
|   |   | o | o |
+---+---+---+---+
|   |   |   | o |
+---+---+---+---+
+---+---+---+---+
| o | o |   |   |
+---+---+---+---+
| X | X | X |   |
+---+---+---+---+
|   |   | o |   |
+---+---+---+---+
|   |   | o | o |
+---+---+---+---+
+---